# Детекция на примере `yolov5`

## Скачиваем датасеты

In [32]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="vVzgCl7OcCV2hguH2FcC")
project = rf.workspace("cs474-ug2-vehicle-detection").project("object-detection-um7ee")
version = project.version(3)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Object-Detection-3 in yolov5pytorch:: 100%|████████████████████████████████████████████████████████████████████████████████████████████████| 7882/7882 [00:01<00:00, 5042.55it/s]


Дополнительный датасет

In [31]:
# !pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="vVzgCl7OcCV2hguH2FcC")
project = rf.workspace("object-detection-gstrg").project("car-detection-fuller")
version = project.version(1)
dataset = version.download("yolov5")

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to car-detection-fuller-1 in yolov5pytorch:: 100%|██████████████████████████████████████████████████████████████████████████████████████████| 17325/17325 [00:03<00:00, 5363.43it/s]


In [30]:
%cd ..
!ls

/workspace/Daniil
data.yaml  datasets  hyp.yaml  yolov5  yolov5.ipynb


## Подготовка

Клонируем репозиторий [`ultralytics/yolov5`](https://github.com/ultralytics/yolov5)

In [7]:
!git clone https://github.com/ultralytics/yolov5.git

fatal: destination path 'yolov5' already exists and is not an empty directory.


In [8]:
%cd yolov5

/workspace/Daniil/yolov5


using dhist requires you to install the `pickleshare` library.


In [9]:
%ls

CITATION.cff     README.zh-CN.md  detect.py   pyproject.toml    tutorial.ipynb
CONTRIBUTING.md  benchmarks.py    export.py   requirements.txt  utils/
LICENSE          classify/        hubconf.py  segment/          val.py
README.md        data/            models/     train.py


In [10]:
!python -m pip install -r yolov5/requirements.txt

ERROR: Could not open requirements file: [Errno 2] No such file or directory: 'yolov5/requirements.txt'


## Пробуем запустить

Сначала попробуем запустить код с их обученной моделью, а потом будем пробовать запускать своё обучение

In [11]:
# %cat data/coco128.yaml

In [12]:
%ls data/coco128.yaml

data/coco128.yaml


In [13]:
!python val.py --weights yolov5s.pt --data data/coco128.yaml --imgsz 640 --device 0  # cuda:0

val: data=data/coco128.yaml, weights=['yolov5s.pt'], batch_size=32, imgsz=640, conf_thres=0.001, iou_thres=0.6, max_det=300, task=val, device=0, workers=8, single_cls=False, augment=False, verbose=False, save_txt=False, save_hybrid=False, save_conf=False, save_json=False, project=runs/val, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-306-gb599ae42 Python-3.10.12 torch-2.2.0a0+81ea7a4 CUDA:0 (Tesla V100-SXM2-32GB, 32501MiB)

100%|██████████████████████████████████████| 14.1M/14.1M [00:00<00:00, 17.4MB/s]

Fusing layers... 
YOLOv5s summary: 213 layers, 7225885 parameters, 0 gradients, 16.4 GFLOPs

Dataset not found ⚠️, missing paths ['/workspace/Daniil/datasets/coco128/images/train2017']
100%|██████████████████████████████████████| 6.66M/6.66M [00:00<00:00, 11.8MB/s]
Dataset download success ✅ (4.4s), saved to /workspace/Daniil/datasets
100%|████████████████████████████████████████| 755k/755k [00:00<00:00, 2.44MB/s]
val: Scanning /workspace/Daniil/datasets/coco128/labels

Датасет coco128 был автоматически скачан, посмотрим на структуру датасета

In [14]:
# %ls ../datasets/coco128

In [15]:
# %ls ../datasets/coco128/images/train2017

In [16]:
# %ls ../datasets/coco128/labels/train2017/

Посмотрим на то, что представляют из себя файлы с аннотациями

In [17]:
# %cat  ../datasets/coco128/labels/train2017/000000000009.txt

Формат разметки, который здесь используется, называется `yolo`-форматом:

`[cls, x_center / img_w, y_center / img_h, w / img_w, h / img_h]`

Есть и другие популярные форматы разметки, например, `coco`-формат. А вот что значат цифры в начале:

```text
45: bowl
46: banana
47: apple
48: sandwich
49: orange
50: broccoli
```

In [18]:
# import cv2
# import matplotlib.pyplot as plt
# from google.colab.patches import cv2_imshow

In [19]:
# img = cv2.imread('../datasets/coco128/images/train2017/000000000009.jpg')
# cv2_imshow(img)
# plt.show()

In [20]:
# !python detect.py --weights yolov5s.pt --source ../datasets/coco128/images/train2017/000000000009.jpg --name some_name

In [21]:
# img = cv2.imread('runs/detect/some_name/000000000009.jpg')
# cv2_imshow(img)
# plt.show()

Также можно запустить `detect.py` не только на отдельном изображении, но и на папке с изображениями, на потоке, на изображении из сети и т.п.

## Как запустить своё обучение

Чтобы запустить своё обучение, надо:

- Подготовить датасет в `yolo`-формате, создать свой конфиг с путями до данных вида `data/my_data.yaml`. Примеры того, как организовывать файлы в датасет, можно посмотреть в [`data/coco.yaml`](https://github.com/ultralytics/yolov5/blob/master/data/coco.yaml) и [`data/VOC.yaml`](https://github.com/ultralytics/yolov5/blob/master/data/VOC.yaml)
- Создать файлик с конфигом, в котором будут прописаны параметры для тренировки, вида `data/hyps/my_hyps.yaml`. Для начала можно попробовать запустить со стандартным [конфигом](https://github.com/ultralytics/yolov5/blob/master/data/hyps/hyp.scratch-low.yaml)
- Стартовать с весов `yolov5s`, если будете тренировать yolo small; `yolov5n`, если yolo nano, и т.д.
![](https://github.com/ultralytics/yolov5/releases/download/v1.0/model_comparison.png)
- Лучше передавать в `--name` какое-то адекватное название, которое описывает суть эксперимента, чтобы не перепутать потом их между собой
- Графики можно смотреть во время обучения, для этого надо запустить в терминале что-то типа:

  ` tensorboard --logdir ./runs/train/`

  и перейти по ссылке в браузере

Пример:

In [22]:
# !cd ..
# !rm -r yolov5
# !move "Object-Detection-3" "yolov5"

In [23]:
# %cd yolov5
!ls

CITATION.cff	 __pycache__	export.py	  runs		  val.py
CONTRIBUTING.md  benchmarks.py	hubconf.py	  segment	  yolov5s.pt
LICENSE		 classify	models		  train.py
README.md	 data		pyproject.toml	  tutorial.ipynb
README.zh-CN.md  detect.py	requirements.txt  utils


Создаём необходимые yaml файлы

In [24]:
import yaml


hyp = {
    'lr0': 0.001,
    'lrf': 0.01,
    'momentum': 0.937,
    'weight_decay': 0.0005,
    'warmup_epochs': 3.0,
    'warmup_momentum': 0.8,
    'warmup_bias_lr': 0.1,
    'box': 0.05,
    'cls': 0.5,
    'cls_pw': 1.0,
    'obj': 1.0,
    'obj_pw': 1.0,
    'iou_t': 0.4,
    'anchor_t': 4.0,
    'fl_gamma': 0.0,
    'hsv_h': 0.015,
    'hsv_s': 0.3,
    'hsv_v': 0.2,
    'degrees': 20.0,
    'translate': 0.1,
    'scale': 0.5,
    'shear': 0.0,
    'perspective': 0.0,
    'flipud': 0.5,
    'fliplr': 0.5,
    'mosaic': 1.0,
    'mixup': 0.0,
    'copy_paste': 0.0
}

    
with open("hyp.yaml", "w") as f:
    yaml.dump(hyp, f)

In [25]:
# print(/kaggle/working/yolov5/Object-Detection-3/data.yaml)

In [ ]:
import os
print(len(os.listdir('/car-detection-fuller-1/train/images')))
print(len(os.listdir('/Object-Detection-3/train/images')))

In [ ]:
data = {
    'train': ['../Object-Detection-3/train/images', '../car-detection-fuller-1/train/images'],
    'val': ['../Object-Detection-3/valid/images'],
    'nc': 1,
    'names': ['vehicle']
}


with open("data.yaml", "w") as f:
    yaml.dump(data, f)

In [ ]:
import wandb
wandb.login(key='e9b7faf1d7a0ab67389ae0d497c3cac4b74e93a7')
wandb.init()

In [ ]:
!python train.py --data /kaggle/working/yolov5/data.yaml --hyp /kaggle/working/yolov5/hyp.yaml \
--weights yolov5s.pt --epochs 200 --batch-size 32 --optimizer SGD \
--name lab2_small  --imgsz 1280

Сохраняем результаты

Какие ещё ключи есть у `train.py`? Идём в скрипт и находим вот такую часть кода, читаем её:

```python
def parse_opt(known=False):
    parser = argparse.ArgumentParser()
    parser.add_argument('--weights', type=str, default=ROOT / 'yolov5s.pt', help='initial weights path')
    parser.add_argument('--cfg', type=str, default='', help='model.yaml path')
    parser.add_argument('--data', type=str, default=ROOT / 'data/coco128.yaml', help='dataset.yaml path')
    parser.add_argument('--hyp', type=str, default=ROOT / 'data/hyps/hyp.scratch-low.yaml', help='hyperparameters path')
    parser.add_argument('--epochs', type=int, default=100, help='total training epochs')
    parser.add_argument('--batch-size', type=int, default=16, help='total batch size for all GPUs, -1 for autobatch')
    parser.add_argument('--imgsz', '--img', '--img-size', type=int, default=640, help='train, val image size (pixels)')
    parser.add_argument('--rect', action='store_true', help='rectangular training')
    parser.add_argument('--resume', nargs='?', const=True, default=False, help='resume most recent training')
    parser.add_argument('--nosave', action='store_true', help='only save final checkpoint')
    parser.add_argument('--noval', action='store_true', help='only validate final epoch')
    parser.add_argument('--noautoanchor', action='store_true', help='disable AutoAnchor')
    parser.add_argument('--noplots', action='store_true', help='save no plot files')
    parser.add_argument('--evolve', type=int, nargs='?', const=300, help='evolve hyperparameters for x generations')
    parser.add_argument('--bucket', type=str, default='', help='gsutil bucket')
    parser.add_argument('--cache', type=str, nargs='?', const='ram', help='image --cache ram/disk')
    parser.add_argument('--image-weights', action='store_true', help='use weighted image selection for training')
    parser.add_argument('--device', default='', help='cuda device, i.e. 0 or 0,1,2,3 or cpu')
    parser.add_argument('--multi-scale', action='store_true', help='vary img-size +/- 50%%')
    parser.add_argument('--single-cls', action='store_true', help='train multi-class data as single-class')
    parser.add_argument('--optimizer', type=str, choices=['SGD', 'Adam', 'AdamW'], default='SGD', help='optimizer')
    parser.add_argument('--sync-bn', action='store_true', help='use SyncBatchNorm, only available in DDP mode')
    parser.add_argument('--workers', type=int, default=8, help='max dataloader workers (per RANK in DDP mode)')
    parser.add_argument('--project', default=ROOT / 'runs/train', help='save to project/name')
    parser.add_argument('--name', default='exp', help='save to project/name')
    parser.add_argument('--exist-ok', action='store_true', help='existing project/name ok, do not increment')
    parser.add_argument('--quad', action='store_true', help='quad dataloader')
    parser.add_argument('--cos-lr', action='store_true', help='cosine LR scheduler')
    parser.add_argument('--label-smoothing', type=float, default=0.0, help='Label smoothing epsilon')
    parser.add_argument('--patience', type=int, default=100, help='EarlyStopping patience (epochs without improvement)')
    parser.add_argument('--freeze', nargs='+', type=int, default=[0], help='Freeze layers: backbone=10, first3=0 1 2')
    parser.add_argument('--save-period', type=int, default=-1, help='Save checkpoint every x epochs (disabled if < 1)')
    parser.add_argument('--seed', type=int, default=0, help='Global training seed')
    parser.add_argument('--local_rank', type=int, default=-1, help='Automatic DDP Multi-GPU argument, do not modify')

    # Logger arguments
    parser.add_argument('--entity', default=None, help='Entity')
    parser.add_argument('--upload_dataset', nargs='?', const=True, default=False, help='Upload data, "val" option')
    parser.add_argument('--bbox_interval', type=int, default=-1, help='Set bounding-box image logging interval')
    parser.add_argument('--artifact_alias', type=str, default='latest', help='Version of dataset artifact to use')

    return parser.parse_known_args()[0] if known else parser.parse_args()
```



(10 эпох - это достаточно мало, да и сама эпоха у вас будет занимать гораздо больше времени, ведь coco128 - это маленький датасет для отладки)

Приятная опция: прерванные эксперименты можно возобновлять (если нечаянно нажали ctrl + c или потерялось соединение с сервером). Так выглядит код повторного запуска обучения, если вы прервали его:

In [ ]:
# !python train.py --data data/Objects365.yaml --hyp data/hyps/hyp.scratch-low.yaml --weights yolov5m.pt \
# --epochs 5 --batch-size 32 --optimizer SGD --name example_Objects365 --resume runs/train/example_Objects365/weight/last.pt

Провалидировать модель можно при помощи val.py:

In [ ]:
!ls
# !rm -r yolov5

In [ ]:
!git clone https://github.com/DaniilMako/yolov5.git

In [ ]:
!python val.py --weights yolov5/train/lab2_small/weights/best.pt \
--data Object-Detection-3/data.yaml --imgsz 1280

# Тестим

In [ ]:
!python detect.py --weights /kaggle/working/yolov5/yolov5/weights/best.pt \
--source /kaggle/working/yolov5/yolov5/mixkit-daytime-city-traffic-aerial-view-56-medium.mp4 \
--name test_27_04  --imgsz 1280

In [ ]:
!zip -r /kaggle/working/runs.zip /kaggle/working/yolov5/runs

In [ ]:
!ls